In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, optimizers,models
from tensorflow.keras.applications import ResNet50,InceptionV3,EfficientNetB0,VGG19
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [3]:
train_data="/kaggle/input/d/akhilraj007/covid-detection/Data/train"
test_data="/kaggle/input/d/akhilraj007/covid-detection/Data/test"
image_generator=ImageDataGenerator(rescale=1.0/255,validation_split=0.2)
train_generator = image_generator.flow_from_directory(batch_size = 32, directory= train_data, shuffle= True, target_size=(256,256), class_mode= 'categorical', subset="training")
val_generator = image_generator.flow_from_directory(batch_size = 32, directory= train_data, shuffle= True, target_size=(256,256), class_mode= 'categorical', subset="validation")
test_generator=image_generator.flow_from_directory(directory= test_data, shuffle= True, target_size=(256,256), class_mode= 'categorical')
print(train_generator.class_indices)

Found 4116 images belonging to 3 classes.
Found 1028 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.
{'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2}


In [4]:
covid19_img,normal_img,pneumonia_img=0,0,0
for i in train_generator.labels:
    if i==0:
        covid19_img+=1
    elif i==1:
        normal_img+=1
    else:
        pneumonia_img+=1
total_imgs=covid19_img+normal_img+pneumonia_img
print('Covid_images:'+str(covid19_img)+'   Normal_images:'+str(normal_img)+'   Pneumonia_images:'+str(pneumonia_img))
class_weights={0:(1/covid19_img)*(total_imgs/3),1:(1/normal_img)*(total_imgs/3),2:(1/pneumonia_img)*(total_imgs/3)}
print(class_weights)

Covid_images:368   Normal_images:1013   Pneumonia_images:2735
{0: 3.7282608695652173, 1: 1.3543928923988153, 2: 0.5016453382084095}


In [5]:
basemodel=ResNet50(include_top=False,weights="imagenet",input_shape=(256,256,3))
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(256, activation  = 'relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(128, activation = 'relu')(headmodel)
headmodel = Dropout(0.2)(headmodel)
headmodel = Dense(3, activation= 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ["accuracy"])
mcp_save =tf.keras.callbacks.ModelCheckpoint('training_1/cp.ckpt', save_best_only=True, monitor='val_accuracy', mode='max')
model.fit(train_generator,validation_data=val_generator,steps_per_epoch=train_generator.n//32,validation_steps=val_generator.n//32,class_weight=class_weights,callbacks=[mcp_save],epochs=10)

94765736/94765736 [==============================] - 4s 0us/step
Epoch 1/10
128/128 [==============================] - 224s 1s/step - loss: 0.5638 - accuracy: 0.8328 - val_loss: 44.4071 - val_accuracy: 0.0898
Epoch 2/10
128/128 [==============================] - 144s 1s/step - loss: 0.2594 - accuracy: 0.9011 - val_loss: 3.5613 - val_accuracy: 0.0898
Epoch 3/10
128/128 [==============================] - 143s 1s/step - loss: 0.3003 - accuracy: 0.8866 - val_loss: 2.3562 - val_accuracy: 0.0898
Epoch 4/10
128/128 [==============================] - 143s 1s/step - loss: 0.2282 - accuracy: 0.9114 - val_loss: 6.4961 - val_accuracy: 0.0889
Epoch 5/10
128/128 [==============================] - 167s 1s/step - loss: 0.1556 - accuracy: 0.9351 - val_loss: 1.2561 - val_accuracy: 0.5508
Epoch 6/10
128/128 [==============================] - 143s 1s/step - loss: 0.1124 - accuracy: 0.9543 - val_loss: 5.6830 - val_accuracy: 0.1836
Epoch 7/10
128/128 [==============================] - 143s 1s/step - loss: 0

In [6]:
model.load_weights('training_1/cp.ckpt')
model.evaluate(test_generator)

41/41 [==============================] - 36s 883ms/step - loss: 0.5797 - accuracy: 0.8727


[0.579657256603241, 0.8726708292961121]

In [7]:
pred=model.predict(test_generator,verbose=1)
predicted_class=np.argmax(pred,axis=1)
actual_class=test_generator.labels

41/41 [==============================] - 30s 706ms/step


In [8]:
tf.math.confusion_matrix(
    actual_class,
    predicted_class,
    num_classes=3
)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 11,  42,  63],
       [ 35, 113, 169],
       [ 78, 293, 484]], dtype=int32)>

In [9]:
print(classification_report(actual_class, predicted_class,target_names=['Covid19','Normal','Pnemonia'] ))

              precision    recall  f1-score   support

     Covid19       0.09      0.09      0.09       116
      Normal       0.25      0.36      0.30       317
    Pnemonia       0.68      0.57      0.62       855

    accuracy                           0.47      1288
   macro avg       0.34      0.34      0.33      1288
weighted avg       0.52      0.47      0.49      1288



In [10]:
basemodel=InceptionV3(include_top=False,weights="imagenet",input_shape=(256,256,3))
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(256, activation  = 'relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(128, activation = 'relu')(headmodel)
headmodel = Dropout(0.2)(headmodel)
headmodel = Dense(3, activation= 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ["accuracy"])
mcp_save =tf.keras.callbacks.ModelCheckpoint('training_2/cp.ckpt', save_best_only=True, monitor='val_accuracy', mode='max')
model.fit(train_generator,validation_data=val_generator,steps_per_epoch=train_generator.n//32,validation_steps=val_generator.n//32,class_weight=class_weights,callbacks=[mcp_save],epochs=10)

87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/10
128/128 [==============================] - 198s 1s/step - loss: 0.4086 - accuracy: 0.8386 - val_loss: 59.6708 - val_accuracy: 0.2461
Epoch 2/10
128/128 [==============================] - 178s 1s/step - loss: 0.2672 - accuracy: 0.8991 - val_loss: 4.0773 - val_accuracy: 0.7363
Epoch 3/10
128/128 [==============================] - 176s 1s/step - loss: 0.2116 - accuracy: 0.9204 - val_loss: 0.2172 - val_accuracy: 0.9316
Epoch 4/10
128/128 [==============================] - 142s 1s/step - loss: 0.1479 - accuracy: 0.9378 - val_loss: 0.2590 - val_accuracy: 0.9268
Epoch 5/10
128/128 [==============================] - 142s 1s/step - loss: 0.1558 - accuracy: 0.9393 - val_loss: 0.9587 - val_accuracy: 0.8125
Epoch 6/10
128/128 [==============================] - 141s 1s/step - loss: 0.1484 - accuracy: 0.9434 - val_loss: 0.6565 - val_accuracy: 0.7646
Epoch 7/10
128/128 [==============================] - 141s 1s/step - loss: 0

In [11]:
model.load_weights('training_2/cp.ckpt')
model.evaluate(test_generator)

41/41 [==============================] - 31s 761ms/step - loss: 0.1396 - accuracy: 0.9565


[0.13957341015338898, 0.95652174949646]

In [12]:
pred=model.predict(test_generator,verbose=1)
predicted_class=np.argmax(pred,axis=1)
actual_class=test_generator.labels

41/41 [==============================] - 31s 699ms/step


In [13]:
tf.math.confusion_matrix(
    actual_class,
    predicted_class,
    num_classes=3
)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[  8,  45,  63],
       [ 23,  80, 214],
       [ 87, 226, 542]], dtype=int32)>

In [14]:
print(classification_report(actual_class, predicted_class,target_names=['Covid19','Normal','Pnemonia'] ))

              precision    recall  f1-score   support

     Covid19       0.07      0.07      0.07       116
      Normal       0.23      0.25      0.24       317
    Pnemonia       0.66      0.63      0.65       855

    accuracy                           0.49      1288
   macro avg       0.32      0.32      0.32      1288
weighted avg       0.50      0.49      0.49      1288



In [15]:
basemodel=models.Sequential()
basemodel.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3),padding="valid"))
basemodel.add(MaxPooling2D(2,2))
basemodel.add(BatchNormalization())

basemodel.add(Conv2D(64,(3,3),activation='relu',padding="valid"))
basemodel.add(MaxPooling2D(2,2))
basemodel.add(BatchNormalization())

basemodel.add(Conv2D(64,(3,3),activation='relu'))
basemodel.add(MaxPooling2D(2,2))
basemodel.add(BatchNormalization())


headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(256, activation  = 'relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(128, activation = 'relu')(headmodel)
headmodel = Dropout(0.2)(headmodel)
headmodel = Dense(3, activation= 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ["accuracy"])
mcp_save =tf.keras.callbacks.ModelCheckpoint('training_3/cp.ckpt', save_best_only=True, monitor='val_accuracy', mode='max')
model.fit(train_generator,validation_data=val_generator,steps_per_epoch=train_generator.n//32,validation_steps=val_generator.n//32,class_weight=class_weights,callbacks=[mcp_save],epochs=10)

Epoch 1/10
128/128 [==============================] - 139s 1s/step - loss: 0.4341 - accuracy: 0.8404 - val_loss: 1.3888 - val_accuracy: 0.6943
Epoch 2/10
128/128 [==============================] - 134s 1s/step - loss: 0.2380 - accuracy: 0.8994 - val_loss: 1.3007 - val_accuracy: 0.6523
Epoch 3/10
128/128 [==============================] - 118s 926ms/step - loss: 0.1819 - accuracy: 0.9219 - val_loss: 2.1999 - val_accuracy: 0.5645
Epoch 4/10
128/128 [==============================] - 134s 1s/step - loss: 0.1518 - accuracy: 0.9319 - val_loss: 1.5049 - val_accuracy: 0.6787
Epoch 5/10
128/128 [==============================] - 136s 1s/step - loss: 0.1632 - accuracy: 0.9268 - val_loss: 0.7757 - val_accuracy: 0.7715
Epoch 6/10
128/128 [==============================] - 134s 1s/step - loss: 0.1472 - accuracy: 0.9319 - val_loss: 7.2955 - val_accuracy: 0.3105
Epoch 7/10
128/128 [==============================] - 138s 1s/step - loss: 0.1448 - accuracy: 0.9305 - val_loss: 0.6022 - val_accuracy: 0.7

In [16]:
model.load_weights('training_3/cp.ckpt')
model.evaluate(test_generator)

41/41 [==============================] - 30s 723ms/step - loss: 0.1686 - accuracy: 0.9503


[0.1686008721590042, 0.9503105878829956]

In [17]:
pred=model.predict(test_generator,verbose=1)
predicted_class=np.argmax(pred,axis=1)
actual_class=test_generator.labels


41/41 [==============================] - 28s 689ms/step


In [18]:
tf.math.confusion_matrix(
    actual_class,
    predicted_class,
    num_classes=3
) 

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 12,  24,  80],
       [ 24,  80, 213],
       [ 85, 196, 574]], dtype=int32)>

In [19]:
print(classification_report(actual_class, predicted_class,target_names=['Covid19','Normal','Pnemonia'] )) 

              precision    recall  f1-score   support

     Covid19       0.10      0.10      0.10       116
      Normal       0.27      0.25      0.26       317
    Pnemonia       0.66      0.67      0.67       855

    accuracy                           0.52      1288
   macro avg       0.34      0.34      0.34      1288
weighted avg       0.51      0.52      0.52      1288



In [20]:
basemodel=VGG19(include_top=False,weights="imagenet",input_shape=(256,256,3))
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (4,4))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(256, activation  = 'relu')(headmodel)
headmodel = Dropout(0.3)(headmodel)
headmodel = Dense(128, activation = 'relu')(headmodel)
headmodel = Dropout(0.2)(headmodel)
headmodel = Dense(3, activation= 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ["accuracy"])
mcp_save =tf.keras.callbacks.ModelCheckpoint('training_4/cp.ckpt', save_best_only=True, monitor='val_accuracy', mode='max')
model.fit(train_generator,validation_data=val_generator,steps_per_epoch=train_generator.n//32,validation_steps=val_generator.n//32,class_weight=class_weights,callbacks=[mcp_save],epochs=10)

80134624/80134624 [==============================] - 4s 0us/step
Epoch 1/10
128/128 [==============================] - 161s 1s/step - loss: 1.2076 - accuracy: 0.3288 - val_loss: 1.1025 - val_accuracy: 0.0879
Epoch 2/10
128/128 [==============================] - 150s 1s/step - loss: 1.1018 - accuracy: 0.3311 - val_loss: 1.1089 - val_accuracy: 0.2471
Epoch 3/10
128/128 [==============================] - 151s 1s/step - loss: 1.0998 - accuracy: 0.1592 - val_loss: 1.0971 - val_accuracy: 0.2471
Epoch 4/10
128/128 [==============================] - 145s 1s/step - loss: 1.0994 - accuracy: 0.3438 - val_loss: 1.0952 - val_accuracy: 0.2471
Epoch 5/10
128/128 [==============================] - 147s 1s/step - loss: 1.1003 - accuracy: 0.3009 - val_loss: 1.0981 - val_accuracy: 0.2471
Epoch 6/10
128/128 [==============================] - 146s 1s/step - loss: 1.1004 - accuracy: 0.4214 - val_loss: 1.0990 - val_accuracy: 0.2441
Epoch 7/10
128/128 [==============================] - 131s 1s/step - loss: 1.

In [21]:
model.load_weights('training_4/cp.ckpt')
model.evaluate(test_generator)

41/41 [==============================] - 31s 754ms/step - loss: 1.0973 - accuracy: 0.6638


[1.0972983837127686, 0.6638198494911194]

In [22]:
pred=model.predict(test_generator,verbose=1)
predicted_class=np.argmax(pred,axis=1)
actual_class=test_generator.labels

41/41 [==============================] - 29s 688ms/step


In [24]:
tf.math.confusion_matrix(
    actual_class,
    predicted_class,
    num_classes=3
) 

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[  0,   0, 116],
       [  0,   0, 317],
       [  0,   0, 855]], dtype=int32)>

In [26]:
print(classification_report(actual_class, predicted_class,target_names=['Covid19','Normal','Pnemonia'] ))

              precision    recall  f1-score   support

     Covid19       0.00      0.00      0.00       116
      Normal       0.00      0.00      0.00       317
    Pnemonia       0.66      1.00      0.80       855

    accuracy                           0.66      1288
   macro avg       0.22      0.33      0.27      1288
weighted avg       0.44      0.66      0.53      1288



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
